In [1]:
pip install -Uqq fastbook

Note: you may need to restart the kernel to use updated packages.


In case "Add persistent_workers class variable to FakeLoader " error

please downgrade py torch version by using -> `conda install pytorch==1.6.0 torchvision==0.7.0 -c pytorch` command

* https://github.com/fastai/fastai/pull/2878
* https://github.com/lgvaz/faststyle/pull/18


In [1]:
from sklearn.metrics import classification_report

from fastai.tabular.all import *
from fastai.basics import *
from fastai import *

import numpy as np
import pandas as pd
import glob
# torch.set_num_threads(10)

In [10]:
# load the baseline model
#learn.load('TubularLearner.fastAI._stage1')
deployed_path = "/mnt/vdb/thesis/TubularLearner.V.set_10.retrain"
#deployed_path = "TubularLearner.fastAI._stage2.pth"
learner = load_learner(deployed_path)


In [3]:
learner.model

TabularModel(
  (embeds): ModuleList()
  (emb_drop): Dropout(p=0.5, inplace=False)
  (bn_cont): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=400, out_features=1221, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(1221, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1221, out_features=1221, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): BatchNorm1d(1221, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1221, out_features=1221, bias=False)
      (2): ReLU(inplace=True)
    )
    (3): LinBnDrop(
      (0): BatchNorm1d(1221, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=1221, out_

In [11]:
def toPredictDF(path,to_dir,existing_result):
    appended_pkl = []
    for infile in glob.glob(path):
        #print("Read:",infile)
        file_name = os.path.basename(infile)
        result_path=to_dir+"/"+file_name.replace("pkl", "ML.pkl")
        if result_path in existing_result :
        # print("found then skip : " , result)
            continue
        else:
            df = pd.read_pickle(infile)
            ready_df =df[[ "reps"]]
            df_new = ready_df.reps.apply(pd.Series).astype(np.float64)
            df_new.columns = df_new.columns.astype(str)
            dl = learner.dls.test_dl(df_new)
            _preds,_none ,_y = learner.get_preds(dl=dl, with_decoded=True)
            _y =(_preds[:,1] >= 0.35).to(torch.int)
            df.drop(columns=['reps'],inplace =True)
            df['class'] = _y
            #print("Save:",result_path)
            df.to_pickle(result_path)
    print("Complete")

def mergeDF(path,to_dir,file_name):
    appended_data = []
    for infile in glob.glob(path):
        #print(infile)
        data = pd.read_pickle(infile)
        # store DataFrame in list
        appended_data.append(data)
    result_path=to_dir+"/"+file_name
    print("Save:",result_path)
    appended_data = pd.concat(appended_data,ignore_index=true)
    #appended_data['ID'] = appended_data['ID'].astype(int)
    #appended_data.sort_values(by=['ID'], inplace=True)
    appended_data.to_pickle(result_path)
    return appended_data
 

## Poison Frog

In [3]:
existing_result = []
for infile in glob.glob("/mnt/vdb/PoisonFrog/cd100/results/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
toPredictDF("/mnt/vdb/PoisonFrog/cd100/reps/*.pkl","/mnt/vdb/PoisonFrog/cd100/results",existing_result)

In [ ]:
# write result
result_df =mergeDF("/mnt/vdb/PoisonFrog/cd100/results/*.pkl","/mnt/vdb/PoisonFrog/cd100/results","PoisonFrog.len15.MLResult.plk")
result_df

In [10]:
result_df

,ID,class
0,0,1
1,8,1
2,10,1
3,15,1
4,18,0
...,...,...
285,12753702,1
286,12753704,1
287,12753708,1
288,12753717,0


## Rat

In [30]:
existing_result = []
for infile in glob.glob("/mnt/vdb/Rat/cd100/result/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
toPredictDF("/mnt/vdb/Rat/cd100/reps/*.pkl","/mnt/vdb/Rat/cd100/result",existing_result)

In [33]:
# write result
result_df =mergeDF("/mnt/vdb/Rat/cd100/result/*.pkl","/mnt/vdb/Rat/cd100/result","Rat.len15.MLResult.plk")
result_df

Save: /mnt/vdb/Rat/cd100/result/Rat.len15.MLResult.plk


,ID,class
0,0,1
1,1,0
2,2,0
3,3,0
4,5,0
...,...,...
1049,311701,0
1050,311702,0
1051,311703,0
1052,311705,0


## BAT

In [5]:
path = "/mnt/vdb/Bat/cd100/result_5/result_FastAI"
if not os.path.exists(path):
    os.makedirs(path)

existing_result = []
for infile in glob.glob("/mnt/vdb/Bat/cd100/result_5/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [6]:
# Predict
toPredictDF("/mnt/vdb/Bat/cd100/reps/*.pkl","/mnt/vdb/Bat/cd100/result_5/result_FastAI",existing_result)

Complete


In [7]:
# write result
result_df =mergeDF("/mnt/vdb/Bat/cd100/result_5/result_FastAI/*.pkl","/mnt/vdb/Bat/cd100/result_5/result_FastAI","Bat.len10.MLResult.plk")
result_df

Save: /mnt/vdb/Bat/cd100/result_5/result_FastAI/Bat.len10.MLResult.plk


,ID,length,class
0,9,275,1
1,19,47,0
2,32,151,1
3,54,199,1
4,61,152,0
...,...,...,...
74,12568817,51,1
75,12568833,45,1
76,12568849,117,1
77,12568850,174,1


## Red King

In [5]:
existing_result = []
for infile in glob.glob("/mnt/vdb/RedKing/cd100/result/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
toPredictDF("/mnt/vdb/RedKing/cd100/reps/*.pkl","/mnt/vdb/RedKing/cd100/result",existing_result)

In [9]:
# write result
result_df =mergeDF("/mnt/vdb/RedKing/cd100/result/*.pkl","/mnt/vdb/RedKing/cd100/result","RedKing.len10.MLResult.plk")
result_df

Save: /mnt/vdb/RedKing/cd100/result/RedKing.len10.MLResult.plk


,ID,class
0,0,1
1,1,1
2,3,1
3,4,1
4,6,1
...,...,...
25,6492107,1
26,6492113,1
27,6492114,1
28,6492121,1


## Taiwanese Oolong tea

In [37]:
existing_result = []
for infile in glob.glob("/mnt/vdb/Taiwan/cd100/result/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
toPredictDF("/mnt/vdb/Taiwan/cd100/reps/*.pkl","/mnt/vdb/Taiwan/cd100/result",existing_result)

In [39]:
# write result
result_df =mergeDF("/mnt/vdb/Taiwan/cd100/result/*.pkl","/mnt/vdb/Taiwan/cd100/result","Taiwan.len20.MLResult.plk")
result_df

Save: /mnt/vdb/Taiwan/cd100/result/Taiwan.len20.MLResult.plk


,ID,class
0,0,1
1,1,1
2,2,1
3,3,0
4,4,1
...,...,...
107,1058123,0
108,1058124,1
109,1058125,1
110,1058126,0


## komodo

In [10]:
existing_result = []
for infile in glob.glob("/mnt/vdb/komodo/cd100/result/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

17248


In [11]:
# Predict
existing_result = []
toPredictDF("/mnt/vdb/komodo/cd100/reps/*.pkl","/mnt/vdb/komodo/cd100/result",existing_result)

Complete


In [18]:
# write result
result_df =mergeDF("/mnt/vdb/komodo/cd100/result/*.pkl","/mnt/vdb/komodo/cd100/result","komodo.len10.MLResult.plk")
result_df

Save: /mnt/vdb/komodo/cd100/result/komodo.len10.MLResult.plk


,ID,class
0,34,1
1,38,0
2,59,0
3,61,1
4,69,1
...,...,...
33,34495360,1
34,34495366,1
35,34495367,1
36,34495419,1


## German cockroach

In [8]:
path = "/mnt/vdb/DECockroach/cd100/result_5/result_FastAI"
if not os.path.exists(path):
    os.makedirs(path)

existing_result = []
for infile in glob.glob("/mnt/vdb/DECockroach/cd100/result_5/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
# existing_result = []
toPredictDF("/mnt/vdb/DECockroach/cd100/reps/*.pkl","/mnt/vdb/DECockroach/cd100/result_5/result_FastAI",existing_result)

In [10]:
# write result
result_df =mergeDF("/mnt/vdb/DECockroach/cd100/result_5/result_FastAI/*.pkl","/mnt/vdb/DECockroach/cd100/result_5/result_FastAI","DECockroach.len15.MLResult.plk")
result_df

Save: /mnt/vdb/DECockroach/cd100/result_5/result_FastAI/DECockroach.len15.MLResult.plk


,ID,length,class
0,0,17,0
1,1,17,0
2,3,18,0
3,4,30,0
4,5,17,0
...,...,...,...
199,894298,17,0
200,894299,19,0
201,894303,17,0
202,894304,17,0


In [15]:
result_df[result_df["class"]==1]

,ID,length,class
9,10,18,1
10,12,20,1
16,19,17,1
17,20,19,1
24,29,20,1
...,...,...,...
185,894275,31,1
187,894280,17,1
189,894282,32,1
193,894288,17,1


# TransPI

## DECoackroach

In [12]:
path = "/mnt/vdb/DECockroach/transpi/result_10_1/result_FastAI"
if not os.path.exists(path):
    os.makedirs(path)
    
existing_result = []
for infile in glob.glob("/mnt/vdb/DECockroach/transpi/result_10_1/result_FastAI/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [13]:
# Predict
# existing_result = []
toPredictDF("/mnt/vdb/DECockroach/transpi/reps/*.pkl","/mnt/vdb/DECockroach/transpi/result_10_1/result_FastAI",existing_result)

Complete


In [14]:
# write result
result_df =mergeDF("/mnt/vdb/DECockroach/transpi/result_10_1/result_FastAI/*.pkl","/mnt/vdb/DECockroach/transpi/result_10_1/result_FastAI","DECockroach.len15.MLResult.plk")
result_df

Save: /mnt/vdb/DECockroach/transpi/result_10_1/result_FastAI/DECockroach.len15.MLResult.plk


,ID,length,class
0,SPADES.k25.NODE_30088_length_748_cov_5.637621_g20919_i0.p1,103,1
1,SPADES.k25.NODE_3009_length_3585_cov_8.357865_g1871_i0.p1,1099,1
2,SPADES.k25.NODE_30119_length_747_cov_3.958449_g20947_i0.p1,248,1
3,SPADES.k25.NODE_3014_length_3583_cov_14.673974_g1874_i0.p1,1007,1
4,SPADES.k25.NODE_30157_length_746_cov_4.047157_g20976_i0.p1,145,1
...,...,...,...
15419,SPADES.k37.NODE_8370_length_1898_cov_7.698549_g4425_i0.p1,495,1
15420,SPADES.k37.NODE_8452_length_1889_cov_19.348272_g2155_i2.p1,588,1
15421,SPADES.k37.NODE_8457_length_1889_cov_10.151728_g4466_i0.p1,508,1
15422,SPADES.k37.NODE_8488_length_1886_cov_3.239048_g4486_i0.p1,524,1


In [24]:
result_df[result_df["class"]==1]

,ID,length,class
0,SOAP.k25.C372231.p1,102,1
1,SOAP.k25.C373809.p1,104,1
2,SOAP.k25.C379695.p1,95,1
3,SOAP.k25.C382451.p1,116,1
4,SOAP.k25.C383053.p1,104,1
...,...,...,...
54,Velvet.k37.NODE_9552_length_1035_cov_12.333333.p1,102,1
55,Velvet.k37.NODE_9713_length_2207_cov_17.057997.p1,627,1
56,Velvet.k37.NODE_9748_length_894_cov_40.512302.p1,124,1
57,Velvet.k37.NODE_9800_length_726_cov_45.530304.p1,221,1


## Bat

In [15]:
path = "/mnt/vdb/Bat/transpi/result_10_1/result_FastAI"
if not os.path.exists(path):
    os.makedirs(path)


existing_result = []
for infile in glob.glob("/mnt/vdb/Bat/transpi/result_10_1/result_FastAI/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [16]:
toPredictDF("/mnt/vdb/Bat/transpi/reps/*.pkl","/mnt/vdb/Bat/transpi/result_10_1/result_FastAI",existing_result)

Complete


In [17]:
# write result
result_df =mergeDF("/mnt/vdb/Bat/transpi/result_10_1/result_FastAI/*.pkl","/mnt/vdb/Bat/transpi/result_10_1/result_FastAI","Bat.len10.MLResult.plk")
result_df

Save: /mnt/vdb/Bat/transpi/result_10_1/result_FastAI/Bat.len10.MLResult.plk


,ID,length,class
0,SPADES.k41.NODE_512_length_1928_cov_55.923688_g385_i0.p1,631,1
1,SPADES.k41.NODE_5132_length_677_cov_11.740566_g4331_i0.p1,225,1
2,SPADES.k41.NODE_5134_length_676_cov_54.214173_g4333_i0.p1,224,1
3,SPADES.k41.NODE_5166_length_673_cov_61.191456_g4363_i0.p1,218,1
4,SPADES.k41.NODE_5188_length_672_cov_23.599049_g4384_i0.p1,224,1
...,...,...,...
10175,SPADES.k25.NODE_1336_length_1434_cov_31.092974_g1006_i0.p1,392,1
10176,SPADES.k25.NODE_13397_length_390_cov_24.084932_g11821_i0.p1,128,1
10177,SPADES.k25.NODE_13409_length_390_cov_13.263014_g11833_i0.p1,129,1
10178,SPADES.k25.NODE_13420_length_390_cov_8.676712_g11844_i0.p1,130,1


In [28]:
result_df[result_df["class"]==1]

,ID,length,class
1,SOAP.k25.C303708.p1,102,1
2,SOAP.k25.C304032.p1,103,1
3,SOAP.k25.C304284.p1,104,1
4,SOAP.k25.C304386.p1,104,1
5,SOAP.k25.C305552.p1,108,1
...,...,...,...
1000,Velvet.k61.NODE_783_length_328_cov_36.378048.p1,106,1
1001,Velvet.k61.NODE_8470_length_279_cov_15.172043.p1,105,1
1002,Velvet.k61.NODE_9141_length_765_cov_44.260132.p2,233,1
1003,Velvet.k61.NODE_9415_length_1250_cov_53.216000.p1,367,1


# PWM

## BAT -CD100

In [27]:
path = "/mnt/vdb/Bat/pws/cd100/result_9/result_FastAI"
if not os.path.exists(path):
    os.makedirs(path)
    
existing_result = []
for infile in glob.glob(path+"/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

24


In [28]:
toPredictDF("/mnt/vdb/Bat/pws/cd100/reps/*.pkl",path,existing_result)

Complete


In [29]:
result_df =mergeDF(path+"/*.pkl",path,"Bat.len10.MLResult.plk")
result_df

Save: /mnt/vdb/Bat/pws/cd100/result/result_FastAI/Bat.len10.MLResult.plk


,ID,Sequence,length,class
0,9,EDTGFYPSEPMLCSESEEGQVPHSLETLYQSADCSSPSDALIVCIHLLMLESGYIPQGTETKAISMPENWKSGGVYKLQYTHPLCEGGFAALTCVPLGNLVVINATLKINSEIRSVKRLQLLWRCLYLRDFRGENVANIYKDLQKLSRLFKDQLVYPLLAFTRQALNLPDVFGLVVLPLELKLRIFRLLDVRSVLSLSAVCRDLFIASNDQLLWRCLYLRDFRVLIPRVRDTDWKELYKKKHKQRKEAQRIRHMMFLPPSSPHPNPFYPNPLDRK,275,1
1,19,ALGPSLWDRRRSLHLLLQEAFPVAQSLAQVIHHQFQTVSKQGGPLP,47,1
2,32,VGICGSDVHYWQHGRIGDFIVKKPMVLGHEASGTVVKVGSLVKHLQQGDRVAIEPGAPREIDEFCKIGRYNLSPTIFFCATPPDDGNLCRFYKHDANFCYKLPDNVTFEEGALIEPLSVGIHTCRRAGVTLGNKVFVCGAERKEEIEPGAP,151,1
3,54,EKPCNSNQQPLENLVEDTLINYSQFGSPKDHEHNGCKLCQTDRYCEPHDYEYCCPCEWHRTEQDRQLITIENNIKKKACSCEGFPFHEVIQEFLLNKDKLVKAVRYQRPDLLLFQRFTLEKMDWPNHYACEKLLVLLTHYDMIERKLGRRHSNQLQPIRFTQWSEENGVHCFEIEWEKPEKLAVVKDSHSMRLFKNSF,199,1
4,61,AAKFVFRHNDPDHLEKLLKKSNSETPKIVAFETVHSMDGAICPLEELCDVAHQYGALTFVDEVHAVGLYGSRGAGMGERDGIMHKIDIISGTLGKACGCVGGYIAGTRDLVDMVRSYAAGFIFTTSLPPMVLSGALESVRLLKGEEGQALRR,152,1
...,...,...,...,...
17611,12568817,GAWTEVGLPSQDVSVASCNCCRRPMHFELMSEWERSYFGNMGPQYVTTYA,51,1
17612,12568833,LLLPLKVLGFLGGQLSVVVLHQPVHIIIIQLQAMDLEIFSSFAP,45,1
17613,12568849,GGQTPRTAPNPQNPPLPPFWAWSSQNVQFSFKFLILGAEPLDRFLALPEPLPRPRVANPVLVLLVAVAVLEARVNGLWERLLWVVKRLEQLQIPKIHLCRHFGLGRVRMFSFLLSS,117,1
17614,12568850,VEESCTIENNSDSTKPKMAAEVDFGDLELFEAFDHPEESLPKPVHTRFKDGDGDEEDENGVGDTGPRERLRQSEETIERLRAENQELKRKLNILTRPSPKWRQRWILGIWSCSRRLTTQRSRSQSPFTRASRTATATRRTRTGLATRGRGSGSGRARKRSSGSAPRIKNLKEN,174,1


## DECockroach -CD100

In [30]:
path = "/mnt/vdb/DECockroach/pws/cd100/result_9/result_FastAI"
if not os.path.exists(path):
    os.makedirs(path)
    
existing_result = []
for infile in glob.glob(path+"/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

24


In [31]:
toPredictDF("/mnt/vdb/DECockroach/pws/cd100/reps/*.pkl",path,existing_result)

Complete


In [32]:
# write result
result_df =mergeDF(path+"/*.pkl",path,"DECockroach.len15.MLResult.plk")
result_df

Save: /mnt/vdb/DECockroach/pws/cd100/result/result_FastAI/DECockroach.len15.MLResult.plk


,ID,Sequence,length,class
0,0,SSRHLQWDSTLSPHFCF,17,1
1,1,RGRCIRSSCPSSHLGWR,17,0
2,3,YSKFSLCQYYFKFCPYST,18,0
3,4,AADYKPEGLGFDSPWCHWNFSIGNPSGRTM,30,1
4,5,AIPSLPHSRRLWSPHHL,17,1
...,...,...,...,...
26102,894298,IHCEEQQCSVRYVWCRR,17,1
26103,894299,WFDYFVSFHPFNLRNWLA,19,1
26104,894303,WPCTHNGDPCTHFGFSA,17,1
26105,894304,ETCGNQCNCDCADAGIN,17,0


## BAT -Transpi

In [4]:
path = "/mnt/vdb/Bat/pws/transpi/result/result_10_1/result_FastAI"
if not os.path.exists(path):
    os.makedirs(path)
    
existing_result = []
for infile in glob.glob(path+"/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [5]:
toPredictDF("/mnt/vdb/Bat/pws/transpi/reps/*.pkl",path,existing_result)

Complete


In [6]:
result_df =mergeDF(path+"/*.pkl",path,"Bat.len10.MLResult.plk")
result_df

Save: /mnt/vdb/Bat/pws/transpi/result/result_10_1/result_FastAI/Bat.len10.MLResult.plk


,ID,Sequence,length,class
0,SPADES.k25.NODE_10932_length_454_cov_11.496503_g9397_i0.p1,WSFLPSITRKRRYVASVSLHDRIYVIGGYDGRSRLSSVECLDYTADEDGVWYSVAPMNVRRGLAGATTLGDMIYVSGGFDGSRRHTSMERYDPNIDQWSMLGDMQTAREGAGLVVAGGVIYCLGGYDGLNILNSVEKYDPHTGHWTNVTPM,151,1
1,SPADES.k25.NODE_10933_length_454_cov_11.025641_g9398_i0.p1,AQSAPRGICCFSCSRHCFSWALLRFSSSLCTDRPLSNAPRAWGDPFLFLWTGPGLRCGNRDAYPCSASARESPCRRQAGAGPSGRPWHGTVSEGFGRQDHIPNTRQPDRPRTPLHFKMGVPRDYCIKARYASSKLGAKSNPGPCQQGRRLE,151,0
2,SPADES.k25.NODE_10937_length_454_cov_8.202797_g9402_i0.p1,RFVAEVTCLDPLRRSRLEVAKLPMTEHELESENKKWVEFACVTLKNEVYISGGKETQHDVWKYNSSINKWIQIEYLNIGRWRHKMVVLGGKVYVIGGFDGLQRINNVETYDPFHNCWSEAAPLLVHVSSFAATSHKKKLYVIGGGPNGKLA,151,1
3,SPADES.k25.NODE_10954_length_453_cov_22.707944_g9418_i0.p1,PLRRHSPAATAAKMSTGAPAGPSAASGSNRKLQQTQNQVDEVVDIMRVNVDKVLERDQKLSELDDRADALQAGASQFETSAAKLKRKYWWKNCKMWAIGITVVVIIIIIIIDADCRHRCCFVP,124,1
4,SPADES.k25.NODE_1095_length_1556_cov_90.177662_g785_i1.p1,REAMASSSAPPATVPASTAAPGQGFGFASKTKKKHFVQQKVKVFRAADPLVGVFLWGVAHSINELSQVPPPVMLLPDDFKASSKIKVNNHLFHRENLPSHFKFKEYCPQVFRNLRDRFGIDDQDYLVSLTRSPPSESEGSDGRFLISYDRTLVIKEVSSEDIADMHSNLSNYHQYIVKCHGNTLLPQFLGMYRVSVDNEDSYMLVMRNMFSHRLPVHRKYDLKGSLVSREASDKEKVKELPTLKDMDFLNKNQKVYIGEEEKKIFLEKLKRDVEFLVQLKIMDYSLLLGIHDIIRGSEPEEEGPVREVESEGDGDCGLTGPPALVGSYGTSPEGIGSYMLSHRPLGPGEFESFIDVYAIRSAEGAPQKEVYFMGLIDILTQYDAKKKAAHAAKTVKHGAGAEISTVHPEQYAKRFLDFITNIFA,425,1
...,...,...,...,...
10175,Velvet.k61.NODE_783_length_328_cov_36.378048.p1,MSPSQAVYIVPSKGRLIGGLRDTPSYEHFQEDFSTCSLCTFRDLCALPPGGTSRGAPCSLPVPWLAQPASFPSQFCTPFPRKPYRVVNPIPVPSFLNIPQVTHSW,106,0
10176,Velvet.k61.NODE_8470_length_279_cov_15.172043.p1,QDLENAATGDAAVHQRIASLPVEVQEVSLLDKITDKESGERLSKMVEDACMLLADYNGRLAAEIDDRKQLTRMLADFLRCQKEALAEKEHKLEPNQTGSLLFST,105,1
10177,Velvet.k61.NODE_9141_length_765_cov_44.260132.p2,ARAALAMPVKGGTKCIKYLLLGFNFVFWLAGIAVLAIGLWLRFDSQTKSIFDQENSHSSFYTGVYILIGAGALMMLVGFLGCCGAVQESQCMLGLFFMFLLVIFALEIAAAVWGYSHKDEVIKELQEFYTDTYHKLRNKDEPQRETLKAIHYALDCCGIVGGVEQFISDICPKTDILTSITVKPCPEAIKEVFHNKFHIIGAVGIGIAVVMIFGMIFSMILCCAIRRSREMV,233,1
10178,Velvet.k61.NODE_9415_length_1250_cov_53.216000.p1,MSCKPQCSLNHLPTPCARQSAPFRIPAEFLYLVLLLVEGAPFSNFSSSPGRQQKIGIMVGFKATDVPPTATVKFLGAGTAACIADLITFPLDTAKVRLQIQGEKQGAVRAAASVQYRGVLGTILTMARTEGPRSLYNGLVAGLQRQMSFASVRIGLYDSVKQFYTKGSEHAGIGSRLLAGSTTGALAVAVAQPTDVVKVRFQAQARAGGGQRYQSTVDAYKTIAREEGFRGLWKGTSPNIARNAIVNCAELVTYDLIKDALLKANLMTDDLPCHFTSAFGAGFCTTIIASPVDVVKTRYMNSALGQYSSAGQCALTMLQKEGPRAFYKGFMPSFLRLGSWNVVMFVTYEQLKRALMAACTSREAPF,367,1


## DECoackroach - Transpi

In [7]:
path = "/mnt/vdb/DECockroach/pws/transpi/result/result_10_1/result_FastAI"
if not os.path.exists(path):
    os.makedirs(path)
    
existing_result = []
for infile in glob.glob(path+"/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [8]:
toPredictDF("/mnt/vdb/DECockroach/pws/transpi/reps/*.pkl",path,existing_result)

Complete


In [9]:
result_df =mergeDF(path+"/*.pkl",path,"DECockroach.len15.MLResult.plk")
result_df

Save: /mnt/vdb/DECockroach/pws/transpi/result/result_10_1/result_FastAI/DECockroach.len15.MLResult.plk


,ID,Sequence,length,class
0,SPADES.k33.NODE_19516_length_1169_cov_36.485035_g1670_i11.p1,MKATITLKGSRLYTVKCRFNERQFNEFASITKITVFPFLTYIRFNVKLFQFYERQYNENFTITKLKCFPYMMAKSCICELYLFGLINNKYVINEVSTVFCSPLFATTRQVYCPQHRLVTPIVGVRRVCCVPTGSALETQILTLPSVRNKPTFQFSSESRTVTMSTNVKKRKQFSLNARKG,181,1
1,SPADES.k33.NODE_19540_length_1168_cov_3.844053_g11217_i0.p1,MTSKMTDGYVDVNGTPTYVITWGKWVEDKLDDSTEEIILLITGNPGLADYYIPFLTKLHSQLGIPIWAVSLAGHVLPPAPSDQRLPSWDKDPELYNLNGQVKHKMLFIEKYVPPHLRMIMLGHSIGAKMIIEVLRNTEIRSRVTKTYLLFPTIERMAESTNGKLMTGVIKHILPVIFFLSWFFNLLPLVVKKFLVGLQFIVRRIPTYHVAPTLKLINPTVIQNVFYLALDEMEKVKDLDDKLISELGKILFLYYGTTDGWVPVTYWKDMTQNHPEVESTLCTKKIDHAFVLRYSNEMADILTDLINEHRRYDE,314,1
2,SPADES.k33.NODE_19570_length_1166_cov_17.372462_g11236_i0.p1,MNLYSPSDETVELVRKEFKLNDRKIKEALDHLKQWLEMQPHLPNKEENSRLERWLINCKNNLERTKETIDIYYSLKTLSPEIMSGRDSSKPWFKEVTKYGYAAVMPELTLEGDRISVYGVFTDDASTFVVEDMMRLMLMCNDVRLFEDYYLKNIFILDLKNLTMGHVMKSTPTLLKKYILFIMKGFKARIRSMHFINVPAFVDIVMGLIKPHLSKKIVSRIHIYGSDYTKLYNDVPRHLLPIELGGKSGSVNEHFEHFQQIIKSYHDWFMATENLKADESRRPKGQVSCNEMFGIQGSFRNLDVD,306,1
3,SPADES.k33.NODE_19634_length_1163_cov_5.649558_g11278_i0.p1,AIEEQKSINEPLTVKPITSTTSYNKNEEQKRDTGSDESQSVEDVPKFLPTPVAETSSLATTIITKDTDRSVRQTSVTETTTDMSTTSTPESVTQDNIKGIKAKRETASRQFSLDSDTSARVARQTGVDSGSNEYIPDARVARQTAIDSGSNEYFPDAPARVARQTAVDSGSNEYFPDAPARVARQTAVDSGSIEFFPDAPARVARQTDTQQTTEQTTTVIATTSTGNPKDVKRQTDDNVQKPTTLETTTQNKPAVSPPLVSNLPASRPSRDVLGFYQADRMPSAAVTSSTPPEQEQREKRDVESSENKPETKNNPALKRRLSTIPEEPSARMKRSAKAQGLEYRRPQPGGPGNNRPGFGNNRPQPRPGNNRPGGPGNNRPGGPGYNR,387,1
4,SPADES.k33.NODE_19635_length_1163_cov_3.203540_g9748_i2.p1,MGEKGERQFKLLPKIINGGIAGIIGVSCVFPLDLVKTRLQNQQIGPNGECQYKSMLDCFRKTYRAEGYFGMYRGSAVNIVLITPEKAIKLAANDFFRHHLRTSSGTLPLTREMAAGGLAGLFQIIVTTPMELLKIQLQDAGRTAVKEGAAAVQRQTALGITMELLRTKGILGLYKGVGATAFRDVSFSVVYFPLFAHLNALGPRKKDGSGEAVFWCSLLAGIGAGATSAFVVNPFDVVKTRLQLLKRAEGERAYSGITDAFVTTLKNEGITAFFRGAACRMIVIAPLFGIAQMVYFLGVAETIMGMKRS,310,1
...,...,...,...,...
15419,TransABySS.k25.S131726.p1,MIQVDESDPVGEVEPSFPYRDVVIKRGVDMKEQYDLQSELGRGKFGTVYRCQEKSTGLQLAAKFVATPKREDRRNVEREIEIMRLLQHPRLIQLYDAFDNGKVMCVVLELIEGGELFERVIDDDFILTEKSCTVFMRQICEGIEFIHKQNILHLDMKPENILCLTKTGNRIKIIDFGLARKYDPQKKLQVLFGTPEFVAPEVVNFDQISYGTDMWSVGVICYVLLSGLSPFMGETDIETMANVTIGKYDFDDEVFNEISNDAKDFIQQLLVKDKERRYTASQCLMHPWLRRKPKPTPPVAEELDVAKDNLRFFVERWNEHPNSPYLFDTSCHIISPCVYMSPDIETVPSQNSLRGLSPSPCSSLSVSISSSPDSIYEHELGKSPEESSNTSFFHYTFEHKQPPPPKYHLERLQTFERRASDSSCVMRRSGDTAARVNLADEIRKLSDKLFQMASMHGPTSDFENNNVDEHQGPSTAPFSSVNFHPSVTPVYKHSFTTTSQLPPPISSRRESLQDDGIPGVRRPKFRFSNMNRDVPIGSPAPPTNFTSFMVANTTGISPCSSRNSSDTLSSTPYNSQSSASSSHSAPPSPRHSPEAN...,677,1
15420,TransABySS.k25.S131831.p1,MPKSKRDKKVSLTKTKKKNVAFKQQLVDEIRRCVEDYKTIFVFSVQNMRNAKLKDLRSEWRHSRFFFGKNKVMALGLGKTKETEVHDKLHKLSKHLSGQCGLLFTNKEEEEVLTWFESYSEADFARSGNVATETVILPAGPLSQFPHSIEPHLRQLGMPTSLQKGVVTLREEFRVCTVDQPLNPEQAKILKLLGNPMAEFKITMKCMWSKDGTFRKYKSKGKETTKNEENIEPIMDVEMEGEGNAEVDVEN,252,1
15421,TransABySS.k25.S131939.p1,MSKKRKICEENRSFQNTWENEFLFINNKDNKPQCLVCLQVLSVHKEYNIKRHYTQHHAAKFKEYVGEARSTIVSDFKRKVSAQKSIFTKSTSTQKKALLASFVVSAEIAKAKKPCSDGEFVKRCSVKMASALGEQKIASEFENVPLSKQTVTRRIEEIGSHLVNKLKTIVADSVYFSLALDESTDVTDTAQLLIFIRTVSKDFVVHEELLGLEPLHNTTKGQDIYDAVMKVLNIYAYGDNNRLSVIVTDGAKSMRGVQTGLVGLLRKTGINCPALHCIIHQEALCGKILDMEVVMKTVIKCVNVVKGGNRALMHRKFLSFLTEMETEYHDIVLFTQVRWLSRGNCLKRFFALRKEIKQFLEENEKTDRELVTALSDSNFNCLLAFLTDITSHMNVLNLKLQGRNQTISDLVGHLDGFISRLKVFKSTLAKGNFVHFPCCKELMEEEIFDIDFQFSAEMLDKLLMQFSERFSDFNDMRSQLILFNNPINCDISEQEDTIQLELCDLQSDPFFQSRVEVGIDFWKLVPREKYPTLCDFSLRVASMFGSTYICECTFSTMKAVKSSQRSRLTDSHLHDFLRISSSSLTIDFGELVDASD...,603,1
15422,TransABySS.k25.S132027.p1,MWCKTAVRNLYKSLHRQCIHTSVTCRLPLVPIVIEQTGRGERAYDIYSRLLKERIICVMGPISDDLSSLVVAQLLFLQSESSKKPVHMYINSPGGSVTAGLGIYDTMQYILPPIATWCVGQACSMASLLLAAGAPGMRHSLPNARIMIHQPSGGVQGQATDIQIQAEEIIKLKRQINGLYVKHTGLALDKIEMSMERDKFMSPLDAKEFGIIDKILDHPPKHGEKMENSEQMSNAANV,239,1
